### **Data Reading**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from configs.paths import abfss_path, checkpoint_path

In [0]:
df = spark.read.format("parquet")\
    .load(abfss_path("bronze", "products"))

In [0]:
df = df.drop("_rescued_data")

In [0]:
df.createOrReplaceTempView("products")

In [0]:
spark.sql("USE CATALOG databricks_cata")
spark.sql("USE bronze")


Define Discount Function for Price Adjustment in SQL - already there in bootstrap.sql

In [0]:
df = df.withColumn("discounted_price", expr("discount_func(price)"))

Create Uppercase Transformation Function for Brand Name - already there in bootstrap.sql

In [0]:
df.write.format("delta").mode("overwrite").save(abfss_path("silver", "products"))

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS databricks_cata.silver.products_silver
USING DELTA
LOCATION '{abfss_path("silver", "products")}'
""")